In [1]:
import pandas as pd
import os

Dataset A => hate_speech_portuguese
Dataset B => OLID-BR

### Dataset A

In [50]:
from datasets import load_dataset

dataset = load_dataset("hate_speech_portuguese")

Found cached dataset hate_speech_portuguese (C:/Users/vinicius/.cache/huggingface/datasets/hate_speech_portuguese/default/1.0.0/20ad7529b5939c566862ef7d6753aa52f92c45aed182decf84abec62c7894062)
100%|██████████| 1/1 [00:00<00:00, 499.56it/s]


In [71]:
tweets_train = dataset['train']['text']
labels_train = dataset['train']['label']

# Criar o dataframe
df_A = pd.DataFrame()

# Adicionar a coluna "tweet" ao dataframe
df_A['tweet'] = tweets_train

# Adicionar a coluna "hatespeech" ao dataframe
df_A['hatespeech'] = labels_train

size = df_A.shape

# Exibir o tamanho do DataFrame
print('Número de linhas:', size[0])
print('Número de colunas:', size[1])

count_tags = df_A['hatespeech'].value_counts()
count_tags

Número de linhas: 5670
Número de colunas: 2


0    3882
1    1788
Name: hatespeech, dtype: int64

### Dataset B

In [63]:
dataset_path = "dataset-v2"

csv_list = []

for item in os.listdir(dataset_path):
    if item.endswith(".csv"):
        csv_list.append(item)


dataframes = [pd.read_csv(os.path.join(dataset_path, item)) for item in csv_list]

In [64]:
dataframes_merged = pd.concat([dataframes[0], dataframes[2]])

In [69]:
# Filtrar as colunas do dataframes_merged
df_B = dataframes_merged[['text', 'is_offensive']].copy()

# Renomear as colunas no dataframe existente
df_B.rename(columns={'text': 'tweet', 'is_offensive': 'hatespeech'}, inplace=True)

# Substituir os valores na coluna "hatespeech"
df_B['hatespeech'] = df_B['hatespeech'].replace({'OFF': 1, 'NOT': 0})

# Obter o tamanho do DataFrame
size = df_B.shape

# Exibir o tamanho do DataFrame
print('Número de linhas:', size[0])
print('Número de colunas:', size[1])

# Contagem dos valores na coluna "hatespeech"
count_tags = df_B['hatespeech'].value_counts()
print(count_tags)


Número de linhas: 6952
Número de colunas: 2
1    5936
0    1016
Name: hatespeech, dtype: int64


### Fusão dos datasets A e B

In [77]:
dataframes_merged = pd.concat([df_A, df_B])
# Limpar os caracteres "\n" na coluna "text"
dataframes_merged['tweet'] = dataframes_merged['tweet'].replace('\n', '', regex=True)
dataframes_merged

,tweet,hatespeech
0,@__andrea__b O cara vive em outro mundoNão no ...,1
1,@_carmeloneto Estes incompetentes não cuidam n...,0
2,@_carmeloneto Os 'cumpanhero' quebraram todas ...,0
3,@_GlitteryKisses é isso não conseguem pensar n...,0
4,@_iglira bom dia macaco branco haha,1
...,...,...
5209,"eu odeio quando fazem isso cmg, e quando fazem...",1
5210,"juliana aprende a escrever!! ""MAIS USER"" kkkk ...",0
5211,"USER USER USER, achei um pouco de RACISMO cham...",0
5212,Um doente me reconheceu e me chamou bem alto n...,1


In [78]:
count_tags = dataframes_merged['hatespeech'].value_counts()
count_tags

1    7724
0    4898
Name: hatespeech, dtype: int64

In [79]:
dataframes_merged.to_csv('dataset-merged/dataset_merged.csv', index=False)